<a href="https://colab.research.google.com/github/LuisaBeccar/proyecto_nlp/blob/main/classification_symptoms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CoderHouse: NLP & deep learning para la Ciencia de Datos
## Comision: 61195 - Nov-Dic 2024
## Luisa Beccar Varela

  ## Clasificación de enfermedades segun descripción de sintomas. <br>



---

## Alcance:
Este modelo podria servir como triage en sistemas de telemedicina para derivar al paciente con el especialista medico adecuado.

---

## Desarrollo del trabajo:

Parto de un dataset tomado de kaggle que guarde en mi github.<br>

Luego de un breve analisis del mismo decidí simplificar la clasificacion: descargue la tabla de enfermedades unicas y genere una sub clasificacion de las enfermedades segun las considere: Dermatologicas, Enfermedades Cronicas, Infecciosas, Traumatologicas o Gastrointestinales. <br>

Así en lugar de 24 enfermedades, reduje el problema a 5 las categorias mencionadas.<br>

Dividi el dataset en set de testeo y entrenamiento. <br>

Luego instancie los modelos de clasificacion: Logistic Regression, Random Forest, Suport Vector Machine Classifier y Gradient Boosting Classifier.<br>

Entrene cada modelo con el x_train transformado con CounterVectorizer considerando trigramas. <br>

Luego predije para cada set de testeo y con eso imprimo el classification report de cada modelo y miro el que mejor accuracy score obtuvo.

In [ ]:
%%capture
# librerias

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/LuisaBeccar/proyecto_nlp/refs/heads/main/Symptom2DiseaseKaggle.csv")
df.drop(columns="Unnamed: 0", inplace=True)
df.rename(columns={"text":"symptoms", "label":"illness"}, inplace=True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   illness   1200 non-null   object
 1   symptoms  1200 non-null   object
dtypes: object(2)
memory usage: 18.9+ KB


In [ ]:
df.symptoms.value_counts()
# veo que hay descripciones repetidas

,count
symptoms,
"I've been feeling extremely scratchy, sick, and worn out. In addition, I've lost weight and have a temperature. My urine is dark, and my skin has turned yellow. Additionally, I have been experiencing stomach pain.",4
I've been feeling really weak in my muscles and my neck has been really stiff. My joints have been swelling up and it's hard for me to move around without feeling stiff. Walking has been really painful too.,3
"I've been constipated and it's really hard to go to the bathroom. When I do go, it hurts and my stool has been bloody. I've also been having some pain in my butt and my anus has been really itchy and irritated.",3
"I've been experiencing stiffness and weakness in my neck muscles recently. Since my joints have grown, it's hard for me to walk without getting stiff. Additionally, walking has been extremely uncomfortable.",2
"Since I've been constipated, using the restroom has been quite challenging. When I do go, it aches and I've had blood in my stool. My anus has been quite itchy and irritating, and I've also been experiencing some discomfort in my butt.",2
...,...
"I've been experiencing severe weariness and a sickly sensation. My throat has been really painful and I've had a fairly severe cough. My nose has been quite stuffy, and I've had significant sinus pressure. I'm really miserable and extremely exhausted.",1
I've been feeling really sick and I've had a lot of fatigue. I've had a really bad cough and my throat has been really sore. I've had a lot of sinus pressure and my nose has been really congested. I just feel really run down and miserable.,1
"I've been quite exhausted and ill. My throat has been quite painful, and I've had a fairly nasty cough. I've got a lot of chills and a pretty high temperature. Simply put, I feel really run down and feeble.",1


In [ ]:
# elimino los duplicados
df.drop_duplicates(inplace=True)
df.symptoms.value_counts()

,count
symptoms,
"I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches.",1
I've recently been dealing with an itching rash on my skin. It's covered in pus-filled pimples and blackheads. My skin has also been quite sensitive.,1
"I have been experiencing intense itching, vomiting, and fatigue. I have also lost weight and have a high fever. My skin has turned yellow and my urine is dark. I have also been experiencing abdominal pain.",1
"Back pain, a productive cough, and muscle weakness have been bothering me. I have issues with dizziness and uneven balance, and my neck hurts.",1
"I have been struggling with back pain, a productive cough, and weakness in my muscles. My neck is sore and I have had problems with dizziness and unsteady balance.",1
...,...
"I can't stop coughing and I'm freezing. My sinuses are completely blocked, and I'm inundated with mucous. My chest hurts, and I'm unable to smell anything. My muscles are also really achy.",1
"I'm coughing nonstop and am really chilly. My mucus production is excessive, and my sinuses are fully clogged. I can't smell anything, and my chest hurts. My muscles are quite painful as well.",1
I can't stop coughing and I feel really cold. My sinuses are all blocked and I have a lot of mucus. My chest hurts and I can't smell anything. My muscles are also really sore.,1


In [ ]:
# cuento cuantas descripciones de cada illness hay
df.illness.value_counts()

,count
illness,
Psoriasis,50
Varicose Veins,50
peptic ulcer disease,50
drug reaction,50
allergy,50
urinary tract infection,50
Hypertension,50
diabetes,50
Fungal infection,50


In [ ]:
enfermedades = df.illness.value_counts().reindex(df.illness.value_counts().index)
enfermedades

,count
illness,
Psoriasis,50
Varicose Veins,50
peptic ulcer disease,50
drug reaction,50
allergy,50
urinary tract infection,50
Hypertension,50
diabetes,50
Fungal infection,50


In [ ]:
# como me parecen muchas categorias y quiero simplificar mi problema voy a subclasificar estas enfermedades yo misma
enfermedades.to_csv("enfermedades.csv")

In [ ]:
#luego de hacer la clasificacion, subo el archivo csv creado
enf_clas = pd.read_csv("https://raw.githubusercontent.com/LuisaBeccar/proyecto_nlp/refs/heads/main/enfermedades_clasificadas.csv")
enf_clas

,illness,clasificacion
0,Psoriasis,Dermatologic
1,Varicose Veins,Dermatologic
2,peptic ulcer disease,Gastrointestinal
3,drug reaction,Dermatologic
4,allergy,Dermatologic
5,urinary tract infection,Infection
6,Hypertension,Cronic Disease
7,diabetes,Cronic Disease
8,Fungal infection,Dermatologic
9,Dengue,Infection


In [ ]:
# mergeo el dataset de las descripciones e illness con el dataset de subclasificaciones para que me ponga una nueva columna con mi subclasificacion segun la ilness
df = pd.merge(df, enf_clas, on="illness", how="left")
# elimino la columna de illness que no voy a usar
df.drop(columns="illness", inplace=True)
df

,symptoms,clasificacion
0,I have been experiencing a skin rash on my arm...,Dermatologic
1,"My skin has been peeling, especially on my kne...",Dermatologic
2,I have been experiencing joint pain in my fing...,Dermatologic
3,"There is a silver like dusting on my skin, esp...",Dermatologic
4,"My nails have small dents or pits in them, and...",Dermatologic
...,...,...
1148,I'm shaking and trembling all over. I've lost ...,Cronic Disease
1149,"Particularly in the crevices of my skin, I hav...",Cronic Disease
1150,I regularly experience these intense urges and...,Cronic Disease
1151,"I have trouble breathing, especially outside. ...",Cronic Disease


In [ ]:
# quiero mirar como quedaron distribuidas mis categorias
clasificadas = df.clasificacion.value_counts().reindex(df.clasificacion.value_counts().index)
clasificadas

,count
clasificacion,
Dermatologic,395
Infection,290
Cronic Disease,242
Gastrointestinal,177
Traumatologic,49


un poco desbalanceadas... las descripciones traumatologicas son poquitas

In [ ]:
# separo las descripciones y las clasificaciones
X = df.symptoms
y = df.clasificacion

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# con stratify espero me tome los conjuntos de datos para corregir el disbalance de clases


(922,) (231,) (922,) (231,)


In [ ]:
# modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

# Fit the model and transform the data
count_vectorizer2 = CountVectorizer(ngram_range=(3,3))

X_train_v = count_vectorizer2.fit_transform(X_train)

modelos = {"LR":LogisticRegression(),
           "RF":RandomForestClassifier(),
           "GB":GradientBoostingClassifier(),
           "SVC":SVC()}

for modelo in modelos:
  modelos[modelo].fit(X_train_v, y_train)
  print(f"Modelo {modelo} entrenado")


Modelo LR entrenado
Modelo RF entrenado
Modelo GB entrenado
Modelo SVC entrenado


In [ ]:
# transformar a array el test set
X_test_v = count_vectorizer2.transform(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# y_pred para cada modelo
for modelo in modelos:
  y_pred = modelos[modelo].predict(X_test_v)
  print(f"Modelo {modelo} predicho")
  print(classification_report(y_test, y_pred))

Modelo LR predicho
                  precision    recall  f1-score   support

  Cronic Disease       0.97      0.78      0.86        49
    Dermatologic       0.74      0.97      0.84        79
Gastrointestinal       1.00      0.80      0.89        35
       Infection       0.94      0.83      0.88        58
   Traumatologic       1.00      0.90      0.95        10

        accuracy                           0.87       231
       macro avg       0.93      0.86      0.88       231
    weighted avg       0.89      0.87      0.87       231

Modelo RF predicho
                  precision    recall  f1-score   support

  Cronic Disease       0.96      0.45      0.61        49
    Dermatologic       0.51      0.99      0.67        79
Gastrointestinal       1.00      0.49      0.65        35
       Infection       0.96      0.47      0.63        58
   Traumatologic       1.00      0.90      0.95        10

        accuracy                           0.66       231
       macro avg       0.89  

In [ ]:
# el mejor modelo
print("El modelo con mejor accuracy es el" , max(modelos, key=lambda key: modelos[key].score(X_test_v, y_test)), "con un accuracy score de", round(max([modelos[key].score(X_test_v, y_test) for key in modelos]),2))

El modelo con mejor accuracy es el LR con un accuracy score de 0.87


##proximamente:<br>
usar validacion cruzada <br>
usar busqueda de hiperparametros para el modelo o los modelos <br>
crear input para probar el modelo con input de syntomas nuevos